# Projet IA01

In [ ]:

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd


heart = pd.read_csv("heart.csv")


In [ ]:
import os
import random
import tensorflow as tf

SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Analyse exploratoire des données

In [ ]:
#distribution des variables catégorielles
plt.figure()
sns.histplot(x='Sex', data=heart)
plt.title('Sex distribution')
plt.show()

plt.figure()
sns.histplot(x='ChestPainType', data=heart)
plt.title('ChestPainType distribution')
plt.show()

plt.figure()
sns.histplot(x='RestingECG', data=heart)
plt.title('RestingECG distribution')
plt.show()

plt.figure()
sns.histplot(x='RestingECG', data=heart)
plt.title('RestingECG distribution')
plt.show()

plt.figure()
sns.histplot(x='ExerciseAngina', data=heart)
plt.title('ExerciseAngina distribution')
plt.show()

plt.figure()
sns.histplot(x='ST_Slope', data=heart)
plt.title('ST Slope distribution')
plt.show()


In [ ]:
#les variables catégorielles ont-elles un impact sur HeartDisease ?

matrice_corr = heart.corr(numeric_only=True)
sns.heatmap(matrice_corr, annot=True)
plt.show()

sns.countplot(x=heart['Sex'], hue=heart['HeartDisease'])
plt.show()

sns.countplot(x=heart['ChestPainType'], hue=heart['HeartDisease'])
plt.show()

sns.countplot(x=heart['RestingECG'], hue=heart['HeartDisease'])
plt.show()

sns.countplot(x=heart['ExerciseAngina'], hue=heart['HeartDisease'])
plt.show()

sns.countplot(x=heart['ST_Slope'], hue=heart['HeartDisease'])
plt.show()

sns.countplot(x=heart['FastingBS'], hue=heart['HeartDisease'])
plt.show()

In [ ]:
#identification des valeurs manquantes et aberrantes :

heart['FastingBS'] = heart['FastingBS'].astype(object)
heart['HeartDisease'] = heart['HeartDisease'].astype(object)

print("Number of Duplicates:", heart.duplicated().sum(), "\n") #nombre de doublon
print("Nombre de NaN : \n" , heart.isna().sum()) #nombre de données manquantes
    

num_df = heart.select_dtypes(include=['int64', 'float64'])

min_values = num_df.describe().loc['min']

print("Minimums des colonnes numeriques : ")
print(min_values)

cat_df = heart.select_dtypes(include=['object'])

print("Valeurs uniques des variables catégorielles : \n")
for col in cat_df.columns :
    unique_value = cat_df[col].unique()
    print(unique_value)

sns.boxplot(y=heart['Age'])
plt.title("Boxplot de l'âge")
plt.show()

sns.boxplot(y=heart['Cholesterol'])
plt.title("Boxplot du Cholesterol")
plt.show()
    
sns.boxplot(y=heart['RestingBP'])
plt.title("Boxplot de la tension")
plt.show()

sns.boxplot(y=heart['MaxHR'])
plt.title("Boxplot du maximum de fréquence cardiaque atteind")
plt.show()

sns.boxplot(y=heart['Oldpeak'])
plt.title("Boxplot de la hauteur de la dépression")
plt.show()


### Traitons maintenant les données manquantes

In [ ]:
#Nettoyage du dataframe
# Supprimer les doublons
heart = heart.drop_duplicates()

# Remplacer les valeurs aberrantes (0 en valeurs manquantes pour les colonnes numériques)
# On peut aussi remplacer par la médianne poru éviter 
heart['Cholesterol'] = heart['Cholesterol'].replace(0, heart['Cholesterol'][heart['Cholesterol'] != 0].mean())
heart['RestingBP'] = heart['RestingBP'].replace(0, heart['RestingBP'][heart['RestingBP'] != 0].mean())


# Encoder LabelEncoder pour Sex et ExerciseAngina
from sklearn.preprocessing import LabelEncoder
le_sex = LabelEncoder()
le_angina = LabelEncoder()
heart['Sex'] = le_sex.fit_transform(heart['Sex'])
heart['ExerciseAngina'] = le_angina.fit_transform(heart['ExerciseAngina'])

# One-Hot Encoding pour ChestPainType, RestingECG et ST_Slope
heart = pd.get_dummies(heart, columns=['ChestPainType', 'RestingECG', 'ST_Slope'], drop_first=True)

print(heart.head())

### Puis les données abérrantes

##### On a décidé de supprimer les données qui sortaient des box plots

In [ ]:
# Sélection des colonnes numériques
numeric_cols = ['Age', 'RestingBP', 'Cholesterol', 'MaxHR', 'Oldpeak']

# Calcul des bornes IQR
Q1 = heart[numeric_cols].quantile(0.25)
Q3 = heart[numeric_cols].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Identification des outliers
outliers_mask = (heart[numeric_cols] < lower_bound) | (heart[numeric_cols] > upper_bound)

# Résultats
print("Nombre d'outliers par variable (après imputation des 0) :")
print(outliers_mask.sum())

total_outlier_rows = outliers_mask.any(axis=1).sum()
total_rows = len(heart)
percent = (total_outlier_rows / total_rows) * 100


# Affichage des bornes
print("\nBornes utilisées :")
print(pd.DataFrame({'Lower': lower_bound, 'Upper': upper_bound}))

##### En cherchant un petit peu et en annalysant plus en profondeur ces données, on remarque que les outliers détectés par les boîtes à moustache ne sont pas très pertinents. En effet, tous sont très significatif sur l'état du patient. De plus en cherchant sur internet, ces données ne sont pas impossibles mais témoignent justement d'un serieux problème.

In [ ]:

for i in numeric_cols: 
    Q1 = heart[i].quantile(0.25)
    Q3 = heart[i].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    if i == 'MaxHR':
        print(heart[(heart[i] < lower) | (heart[i] > upper)][[i,'HeartDisease']].sort_values(by=i,ascending=True).head(5))
    else:
        print(heart[(heart[i] < lower) | (heart[i] > upper)][[i,'HeartDisease']].sort_values(by=i, ascending=False).head(5))

##### On voit clairement ici que ces valeurs identifiées comme aberrantes ont un sens puisqu'elles traduisent presque toutes une maladie.

## Train/Test split + standardisation

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

Y = heart['HeartDisease']
X = heart.drop(['HeartDisease'], axis = 1)

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.3, random_state=0, stratify=Y)

# Question : est-ce qu'on utulise stratify=y qui permet de garder un pourcentage égale de malade dans le teste et train ?
# train_test_split(X,Y, test_size=0.3, random_state=42, stratify=y)


# Centrées réduites de notre dataset
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
heart.head(5)

## KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

Y_train = Y_train.astype(int)
Y_test = Y_test.astype(int)


k_values = range(1, 31)
accuracies_test_man = []
accuracies_train_man = []

accuracies_test_euc=[]
accuracies_train_euc=[]

for k in k_values:
    knn_classifier = KNeighborsClassifier(n_neighbors=k, p=1, metric='minkowski')
    knn_classifier.fit(X_train_scaled, Y_train)
    y_pred_test = knn_classifier.predict(X_test_scaled)
    y_pred_train = knn_classifier.predict(X_train_scaled)
    
    accuracies_test_man.append(accuracy_score(Y_test,y_pred_test))
    accuracies_train_man.append(accuracy_score(Y_train, y_pred_train))

    knn_classifier = KNeighborsClassifier(n_neighbors=k, p=2, metric='minkowski')
    knn_classifier.fit(X_train_scaled, Y_train)
    y_pred_test = knn_classifier.predict(X_test_scaled)
    y_pred_train = knn_classifier.predict(X_train_scaled)
    
    accuracies_test_euc.append(accuracy_score(Y_test,y_pred_test))
    accuracies_train_euc.append(accuracy_score(Y_train, y_pred_train))

# Visualiser la précision en fonction de k

plt.figure(figsize=(12, 7))
plt.plot(k_values, accuracies_test_man, marker='x', label = 'Test_man',color='red')
plt.plot(k_values, accuracies_train_man, marker='x',linestyle='--', label = 'Train_man',color='red',alpha=0.5)
plt.plot(k_values, accuracies_train_euc, marker='x',linestyle='--', label = 'Train_euc', color='blue',alpha=0.5)
plt.plot(k_values, accuracies_test_euc, marker='x', label = 'Test_euc',color='blue')
plt.title('Exactitude du modèle en fonction de k')
plt.xlabel('k')
plt.ylabel('Exactitude')
plt.xticks(k_values)
plt.legend()
plt.grid()
plt.show()

##### On a d'abord utilisé la métrique de l'accuracy pour choisir le meilleur k sauf qu'en réalité, dans le contexte médical il est plus important de maximiser le recall quite à perdre un peu en accurcy. C'est pour cela que l'on a par la suite introduit le F2-Score qui donne deux fois plus de poids au recall qu'à l'accuracy.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import fbeta_score # On importe la métrique
import matplotlib.pyplot as plt
import numpy as np

# Conversion préventive des cibles en entiers
Y_train = Y_train.astype(int)
Y_test = Y_test.astype(int)

# Plages de k
k_values = range(1, 21)

# Listes pour stocker les scores F2
f2_test_man = []
f2_train_man = []
f2_test_euc = []
f2_train_euc = []

for k in k_values:
    # --- Modèle 1 : Manhattan (p=1) ---
    knn_man = KNeighborsClassifier(n_neighbors=k, p=1, metric='minkowski')
    knn_man.fit(X_train_scaled, Y_train)
    
    # Prédictions
    pred_test_man = knn_man.predict(X_test_scaled)
    pred_train_man = knn_man.predict(X_train_scaled)
    
    # Calcul F2 (beta=2 donne 2x plus de poids au Rappel qu'à la Précision)
    f2_test_man.append(fbeta_score(Y_test, pred_test_man, beta=2))
    f2_train_man.append(fbeta_score(Y_train, pred_train_man, beta=2))

    # --- Modèle 2 : Euclidienne (p=2) ---
    knn_euc = KNeighborsClassifier(n_neighbors=k, p=2, metric='minkowski')
    knn_euc.fit(X_train_scaled, Y_train)
    
    # Prédictions
    pred_test_euc = knn_euc.predict(X_test_scaled)
    pred_train_euc = knn_euc.predict(X_train_scaled)
    
    # Calcul F2
    f2_test_euc.append(fbeta_score(Y_test, pred_test_euc, beta=2))
    f2_train_euc.append(fbeta_score(Y_train, pred_train_euc, beta=2))


plt.figure(figsize=(12, 7))

plt.plot(k_values, f2_test_man, marker='o', label='Test Manhattan (F2)', color='red')
plt.plot(k_values, f2_test_euc, marker='o', label='Test Euclidien (F2)', color='blue')

plt.plot(k_values, f2_train_man, marker='x', label='Train Manhattan', linestyle='--', color='red', alpha=0.5)
plt.plot(k_values, f2_train_euc, marker='x', label='Train Euclidien', linestyle='--', color='blue', alpha=0.5)

plt.title('Optimisation du kNN selon le score F2 (Priorité : Détection des malades)')
plt.xlabel('k (Nombre de voisins)')
plt.ylabel('F2-Score')
plt.xticks(k_values)
plt.legend()
plt.grid(True)
plt.show()

print(f2_test_euc[2],f2_test_euc[6],f2_test_euc[18])

In [ ]:
from sklearn.metrics import confusion_matrix

knn_man = KNeighborsClassifier(n_neighbors=9, p=1, metric='minkowski')
knn_man.fit(X_train_scaled, Y_train)
y_pred = knn_man.predict(X_test_scaled)
y_pred_train = knn_man.predict(X_train_scaled)

plt.figure(figsize=(6, 5))
cm = confusion_matrix(Y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=['Sain (Pred)', 'Malade (Pred)'],
            yticklabels=['Sain (Réel)', 'Malade (Réel)'])
plt.title('Matrice de Confusion')
plt.ylabel('Vraie classe')
plt.xlabel('Classe prédite')
plt.show()

print('F2-Score test =',fbeta_score(Y_test,y_pred, beta=2))
print('F2-Score train =',fbeta_score(Y_train,y_pred_train, beta=2))

##### Essayons de chercher si un seuil différent du 'vote majoritaire' changerai quelque chose

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import fbeta_score
import numpy as np
import pandas as pd

# Paramètres à tester
k_values = range(1, 30) # k de 1 à 25
thresholds = np.arange(0.1, 0.95, 0.05) # Seuils de 0.10 à 0.90 par pas de 0.05

best_score = 0
best_params = {}

results = []

print("Recherche du meilleur couple (k, seuil)...")

for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k, p=1) 
    knn.fit(X_train_scaled, Y_train)
    y_prob = knn.predict_proba(X_test_scaled)[:, 1]
    best_f2_for_this_k = 0
    for t in thresholds:
        y_pred_t = (y_prob >= t).astype(int)
        score = fbeta_score(Y_test, y_pred_t, beta=2)
        if score > best_score:
            best_score = score
            best_params = {'k': k, 'threshold': t}
        if score > best_f2_for_this_k:
            best_f2_for_this_k = score
    results.append(best_f2_for_this_k)

print("\n--- RÉSULTAT FINAL ---")
print(f"Le meilleur F2-Score possible est : {best_score:.4f}")
print(f"Obtenu avec : k = {best_params['k']} et Seuil = {best_params['threshold']:.2f}")

# Petit graphique pour voir le potentiel de chaque k (avec son meilleur seuil optimisé)
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
plt.plot(k_values, results, marker='o', color='purple')
plt.title('Meilleur F2-Score possible pour chaque k (après optimisation du seuil)')
plt.xlabel('k (Nombre de voisins)')
plt.ylabel('F2-Score Max')
plt.grid(True)
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt


best_k = best_params['k']
best_thresh = best_params['threshold']

knn_final = KNeighborsClassifier(n_neighbors=best_k, p=1)
knn_final.fit(X_train_scaled, Y_train)


y_probs = knn_final.predict_proba(X_test_scaled)[:, 1]
y_pred_custom = (y_probs >= best_thresh).astype(int)

y_probs_train = knn_final.predict_proba(X_train_scaled)[:,1]
y_pred_train_custom = (y_probs_train >= best_thresh).astype(int)

cm = confusion_matrix(Y_test, y_pred_custom)
plt.figure(figsize=(7, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=['Sain (0)', 'Malade (1)'],
            yticklabels=['Sain (0)', 'Malade (1)'])

plt.title(f'Matrice de Confusion Optimisée\n(k={best_k}, Seuil={best_thresh:.2f})')
plt.xlabel('Prédiction du Modèle')
plt.ylabel('Vraie Réalité')
plt.show()

# 6. Affichage des métriques pour confirmer la réussite
print(classification_report(Y_test, y_pred_custom))


print('F2-Score test =',fbeta_score(Y_test,y_pred_custom, beta=2))
print('F2-Score train =',fbeta_score(Y_train,y_pred_train_custom, beta=2))

## Regression logistique

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve, fbeta_score
import matplotlib.pyplot as plt
import seaborn as sns


model_reg_log = LogisticRegression(C=0.1,max_iter=1000,random_state=42)
model_reg_log.fit(X_train_scaled,Y_train)

y_pred = model_reg_log.predict(X_test_scaled)
y_prob = model_reg_log.predict_proba(X_test_scaled)[:, 1]
y_prob_train = model_reg_log.predict_proba(X_train_scaled)[:, 1]

print('F2-Score =',fbeta_score(Y_test,y_pred, beta=2))
print(f"AUC-ROC Score     : {roc_auc_score(Y_test, y_prob):.4f}")
print(classification_report(Y_test, y_pred))

plt.figure(figsize=(6, 5))
cm = confusion_matrix(Y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=['Sain (Pred)', 'Malade (Pred)'],
            yticklabels=['Sain (Réel)', 'Malade (Réel)'])
plt.title('Matrice de Confusion')
plt.ylabel('Vraie classe')
plt.xlabel('Classe prédite')
plt.show()

fpr, tpr, seuil = roc_curve(Y_test, y_prob)

j = tpr - fpr
ix = np.argmax(j)
meilleur_seuil = seuil[ix]

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'Régression Logistique (AUC = {roc_auc_score(Y_test, y_prob):.2f})')
plt.plot([0, 1], [0, 1], 'r--', label='Aléatoire (AUC = 0.5)') # La diagonale du hasard
plt.scatter(fpr[ix], tpr[ix], marker='o', color='red', s=100, label=(f'Meilleur Seuil{meilleur_seuil:.2f}'))
plt.title('Courbe ROC')
plt.xlabel('Taux de Faux Positifs (FPR)')
plt.ylabel('Taux de Vrais Positifs (TPR / Rappel)')
plt.legend(loc='lower right')
plt.grid()
plt.show()

##### Maintenant que nous avons remarqué que le meilleur seuil est à 0.42 au lieu de 0.5 initiallement, on peut recalculer les prédictions et donc les métrics.

In [ ]:
y_pred_optimal = (y_prob >= meilleur_seuil).astype(int)
y_pred_optimal_train = (y_prob_train >= meilleur_seuil).astype(int)
plt.figure(figsize=(6, 5))
cm = confusion_matrix(Y_test, y_pred_optimal)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=['Sain (Pred)', 'Malade (Pred)'],
            yticklabels=['Sain (Réel)', 'Malade (Réel)'])
plt.title('Matrice de Confusion')
plt.ylabel('Vraie classe')
plt.xlabel('Classe prédite')
plt.show()

print('F2-Score test =',fbeta_score(Y_test,y_pred_optimal, beta=2))
print('F2-Score train =',fbeta_score(Y_train,y_pred_optimal_train, beta=2))
print(classification_report(Y_test, y_pred_optimal))


## Arbres de décision

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

acc_tree_train = []
acc_tree_test = []



max_dep = np.arange(1, 20, 1)

for maxd in max_dep :
    d_tree = DecisionTreeClassifier(max_depth=maxd, random_state=42)
    d_tree.fit(X_train, Y_train)
    y_tree_train_predict = d_tree.predict(X_train)
    y_tree_test_predict = d_tree.predict(X_test)
    acc_tree_train.append(accuracy_score(Y_train, y_tree_train_predict))
    acc_tree_test.append(accuracy_score(Y_test, y_tree_test_predict))

plt.figure()
plt.plot(max_dep, acc_tree_test, marker='o')
plt.plot(max_dep, acc_tree_train, marker='o')
plt.show()

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
import numpy as np

# 1. Définir le modèle (avec un random_state pour la reproductibilité)
tree = DecisionTreeClassifier(random_state=42)

# 2. Définir la grille des hyperparamètres à tester
param_grid = {
    # Tester les profondeurs de 3 à 15, par pas de 2
    'max_depth': np.arange(1, 16, 1), 
    
    # Tester différents critères de division
    'criterion': ['gini', 'entropy'],
    
    # Tester le nombre minimum d'échantillons pour une division
    'min_samples_split': [2, 5, 10] 
}

# 3. Initialiser et lancer la recherche par grille
grid_search = GridSearchCV(
    estimator=tree,
    param_grid=param_grid,
    scoring='recall', # Le critère d'évaluation
    cv=5,               # Utiliser 5 plis de validation croisée
    verbose=1,          # Afficher la progression
    n_jobs=-1           # Utiliser tous les cœurs du processeur
)

# X_train, Y_train sont les données d'entraînement (ne pas utiliser X_test ici)
grid_search.fit(X_train, Y_train)

# 4. Afficher les meilleurs résultats
print("Meilleurs hyperparamètres:", grid_search.best_params_)
print("Meilleur score de CV:", grid_search.best_score_)

# 5. Récupérer le modèle optimal
best_tree = grid_search.best_estimator_

# Utiliser le modèle optimal sur le jeu de test final
final_accuracy = best_tree.score(X_test, Y_test)
print(f"Accuracy final sur le jeu de test: {final_accuracy:.4f}")

# Forêts aléatoires

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix

#en optimisant l'accuracy

forest = RandomForestClassifier(random_state=42)


param_grid = {
    'n_estimators': [200, 400, 600, 800, 1000, 2000],
    'max_depth': [5, 10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': [None, 'sqrt'],
    'bootstrap': [True, False]
}



grid_search = GridSearchCV(
    estimator=forest, 
    param_grid=param_grid, 
    scoring='accuracy',
    cv=5, 
    n_jobs=-1,
    verbose=1
)

# Entraîner le modèle
print("Lancement de l'optimisation pour l'ACCURACY...")
grid_search.fit(X_train, Y_train)

# Afficher les meilleurs résultats
print("\nMeilleurs hyperparamètres trouvés (scoring = 'accuracy') :")
print(grid_search.best_params_)

print("\nMeilleur score d'Accuracy (moyenne CV) :")
print(f"{grid_search.best_score_:.4f}")

# Tester les performances sur l'ensemble de test
y_test_pred = grid_search.best_estimator_.predict(X_test)

print("\nAccuracy finale sur l'ensemble de test :")
print(f"{accuracy_score(Y_test, y_test_pred):.4f}")

print("\nMatrice de confusion :")
print(confusion_matrix(Y_test, y_test_pred))

In [ ]:
#en optimisant le recall

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix
forest = RandomForestClassifier(random_state=42, class_weight='balanced')
# Grille de paramètres à tester (max_depth de 1 à la profondeur maximale trouvée)
param_grid = {
 'n_estimators': [200, 400, 600, 800, 1000, 2000],
 'max_depth': [5, 10, 20, None],
 'min_samples_split': [2, 5, 10],
 'min_samples_leaf': [1, 2, 4],
 'max_features': [None, 'sqrt'],
 'bootstrap': [True, False]
}
# Configurer GridSearchCV avec scoring = 'recall' et cv = 5
grid_search = GridSearchCV(estimator=forest, param_grid=param_grid, scoring='recall', cv=5, n_jobs=-1)
# Entraîner le modèle avec GridSearchCV
grid_search.fit(X_train, Y_train)
# Afficher les meilleurs résultats

print("Meilleurs hyperparamètres trouvés (scoring = 'recall') :")
print(grid_search.best_params_)
print("\nMeilleur score de recall :")
print(grid_search.best_score_)
# Tester les performances sur l'ensemble de test
y_test_pred = grid_search.best_estimator_.predict(X_test)
print("\nAccuracy sur l'ensemble de test :")
print(accuracy_score(Y_test, y_test_pred))

print(confusion_matrix(Y_test, y_test_pred))
# Il est également possible de prendre les meilleur paramètre et d'entraîner un nouveau modèle
# si on ne veut pas utiliser grid_search.best_estimator_

# XGBOOST

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, confusion_matrix, recall_score
import pandas as pd
import numpy as np

#XG Boost ne fonctionne qu'avec des nombres, et non des objets. Il faut donc convertir

# 1. Correction spécifique pour FastingBS (qui doit être un entier 0 ou 1)
# 'coerce' va transformer les valeurs non convertibles en NaN (que XGBoost gère)

X_train_xgb = X_train.copy()
X_test_xgb = X_test.copy()

X_train_xgb['FastingBS'] = pd.to_numeric(X_train_xgb['FastingBS'], errors='coerce').fillna(0).astype(int)
X_test_xgb['FastingBS'] = pd.to_numeric(X_test_xgb['FastingBS'], errors='coerce').fillna(0).astype(int)

# 2. Encodage automatique de TOUTES les autres colonnes textuelles (Sex, ChestPainType, etc.)
# pd.get_dummies transforme "Sex" (M/F) en "Sex_M" (0/1) et "Sex_F" (0/1)
print("Encodage des variables catégorielles (One-Hot Encoding)...")
X_train_xgb = pd.get_dummies(X_train_xgb)
X_test_xgb = pd.get_dummies(X_test_xgb)

# Il faut s'assurer que X_train et X_test ont exactement les mêmes colonnes dans le même ordre
X_train_xgb, X_test_xgb = X_train_xgb.align(X_test_xgb, join='inner', axis=1)

print("Nouvelles dimensions des données :")
print(f"Train : {X_train_xgb.shape}")
print(f"Test  : {X_test_xgb.shape}")

# Calcul du ratio pour équilibrer les classes (équivalent de class_weight='balanced')
count_pos = np.sum(Y_train == 1)
count_neg = np.sum(Y_train == 0)
scale_ratio = count_neg / count_pos if count_pos > 0 else 1

# 1. Configuration du modèle
# On utilise binary:logistic car la cible est binaire (0 ou 1)
xgb = XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False,
    random_state=42,
    scale_pos_weight=scale_ratio # Force le modèle à donner plus de poids aux positifs
)


param_grid = {
    'n_estimators': [100, 200, 300],      # Nombre d'arbres
    'learning_rate': [0.01, 0.05, 0.1],   # Vitesse d'apprentissage 
    'max_depth': [3, 4, 5],               # Profondeur des arbres 
    'subsample': [0.8, 1.0],              # Fraction d'échantillons pour chaque arbre
    'colsample_bytree': [0.8, 1.0]        # Fraction de colonnes par arbre
}

# 3. Configuration de la Recherche par Grille (GridSearchCV)
# cv=5 : Validation croisée à 5 plis (robuste pour cette taille de données)
# scoring='accuracy' ou 'roc_auc' : On cherche à maximiser la précision ici

grid_search = GridSearchCV(
    estimator=xgb,
    param_grid=param_grid,
    cv=5,
    scoring='recall', # Modification ici pour maximiser le Recall
    n_jobs=-1,  # Utilise tous les coeurs du processeur
    verbose=1
)

print("--- Démarrage de l'optimisation des hyperparamètres (Grid Search) ---")

# 4. Entraînement sur vos données d'entraînement existantes
grid_search.fit(X_train_xgb, Y_train)

# 5. Récupération des meilleurs résultats
best_model = grid_search.best_estimator_

print(f"\nMeilleurs hyperparamètres trouvés : {grid_search.best_params_}")
print(f"Meilleur score de validation croisée (Recall) : {grid_search.best_score_:.4f}")

# 6. Évaluation finale sur le jeu de test (X_test, y_test)
y_pred = best_model.predict(X_test_xgb)

# Calcul des métriques
final_recall = recall_score(Y_test, y_pred)
auc = roc_auc_score(Y_test, best_model.predict_proba(X_test_xgb)[:, 1])

print("\n--- PERFORMANCE SUR LE JEU DE TEST ---")
print(f"Recall Final : {final_recall:.4f}")
print(f"AUC Score : {auc:.4f}")
print("\nRapport de Classification détaillé :")
print(classification_report(Y_test, y_pred))

print("Matrice de Confusion :")
print(confusion_matrix(Y_test, y_pred))

## Réseaux de neurones

In [ ]:
from keras.models import Sequential 
from keras.layers import Dense, Input, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.metrics import Recall

model = Sequential([Input(shape=(X_train.shape[1],)),    
                    Dense(16, activation='relu'), 
                    Dense(1, activation='sigmoid') 
                    ]) 

optimizer = Adam(learning_rate = 0.01)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy', Recall(name='recall')])
es = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

model.fit(X_train_scaled,Y_train,epochs=50, batch_size=16, validation_split=0.2, callbacks=[es],verbose=0)



In [ ]:
y_pred_test_proba = model.predict(X_test_scaled)
y_pred_test = (y_pred_test_proba >= 0.5).astype(int)
y_pred_train = (model.predict(X_train_scaled)>= 0.5)
print(f"Réseau de neuronnes Test Set F2-Score {fbeta_score(Y_test,y_pred_test,beta=2)}")
print(f"Réseau de neuronnes Train Set F2-Score {fbeta_score(Y_train,y_pred_train,beta=2)}")
plt.figure(figsize=(6, 5))
cm = confusion_matrix(Y_test, y_pred_test)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=['Sain (Pred)', 'Malade (Pred)'],
            yticklabels=['Sain (Réel)', 'Malade (Réel)'])
plt.title('Matrice de Confusion Test')
plt.ylabel('Vraie classe')
plt.xlabel('Classe prédite')
plt.show()


##### En abaissant la valeur du seuil à 0.3 voir 0.2 on perd un peu en précision par contre on évite des faux négatifs ce qui nous donnes un F2-Score très élevé.

In [ ]:
y_pred_test_proba = model.predict(X_test_scaled)
y_pred_test = (y_pred_test_proba >= 0.2).astype(int)
y_pred_train = (model.predict(X_train_scaled)>= 0.2)
print(f"Réseau de neuronnes Test Set F2-Score {fbeta_score(Y_test,y_pred_test,beta=2)}")
print(f"Réseau de neuronnes Train Set F2-Score {fbeta_score(Y_train,y_pred_train,beta=2)}")
plt.figure(figsize=(6, 5))
cm = confusion_matrix(Y_test, y_pred_test)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=['Sain (Pred)', 'Malade (Pred)'],
            yticklabels=['Sain (Réel)', 'Malade (Réel)'])
plt.title('Matrice de Confusion Test')
plt.ylabel('Vraie classe')
plt.xlabel('Classe prédite')
plt.show()